In [83]:
#!pip install sklearn imutils
#S1: Gather dataset: https://www.kaggle.com/ashishsaxena2209/animal-image-datasetdog-cat-and-panda
#S2: Split the dataset
#S3: Tain the classifier
#S4: Evaluate

In [84]:
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import argparse

In [85]:
class SimplePreprocessor:
    def __init__(self, width, height, inter = cv2.INTER_AREA):
        # store the target image width, height, and interpolation method used when resizing
        self.width = width
        self.height = height
        self.inter = inter
        
    def preprocess(self, image):
        # resize the image to a fixed size, ignoring the aspect ratio
        return cv2.resize(image, (self.width, self.height), interpolation = self.inter )

In [86]:
class SimpleDatasetLoader:
    # preprocessors is a list
    def __init__(self, preprocessors=None):
        # store the image preprocessor
        self.preprocessors = preprocessors
        # If the preprocessors are None, initialize them as an empty list
        if self.preprocessors is None:
            self.preprocessors = []
    # building an image loader
    def load(self, imagePaths, verbose=-1):
        # initialize the list of features and labels
        data = []
        labels = []
        
        for (i, imagePath) in enumerate(imagePaths):
            # load the image and extract the class label assuming that our path has the following format
            #/path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            # check to see if our preprocessors are not None
            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)
            # treat our processed image as a "feature vector"
            # by updating the data list followed by the labels
            data.append(image)
            labels.append(label)
            if verbose > 0 and i > 0 and (i+1)%verbose==0:
                print("[INFO] processed {}/{}".format(i+1, len(imagePaths)))
        return (np.array(data), np.array(labels))

In [87]:
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True, help="path to input dataset")
# ap.add_argument("-k", "--neighbors", type=int, default=1, help="# of nearest neighbors for classification")
# ap.add_argument("-j", "--jobs", type=int, default=1, help="# of jobs for k-NN distance (-1 use all available cores)")
# args=vars(ap.parse_args())
args={}
# '/home/yubao/data/Dataset/Animals/animals/animals/cats/cats_00834.jpg'
args["dataset"] = "/home/yubao/data/Dataset/Animals/animals/animals"
args["neighbors"]=3
args["jobs"]=-1

In [88]:
# grab the list of images that we will be describing
print("[INFO] loading images ...")
imagePaths = list(paths.list_images(args["dataset"]))

# Initialize the image preprocessor, load the dataset from disk and reshape the data matrix
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(imagePaths, verbose=500)
print('[INFO] data shape: ', data.shape)
print('[INFO] label shape: ', labels.shape)
# print('[INFO] data[0]: ', data[0])
print('[INFO] labels[0]: ', labels[0])

# 3072 = 32*32*3
data = data.reshape((data.shape[0], 3072))
print('[INFO] data shape: ', data.shape)
print("[INFO] features matrix: {:.1f}MB".format(data.nbytes/(1024 * 1000.0)))

[INFO] loading images ...
[INFO] processed 500/3000
[INFO] processed 1000/3000
[INFO] processed 1500/3000
[INFO] processed 2000/3000
[INFO] processed 2500/3000
[INFO] processed 3000/3000
[INFO] data shape:  (3000, 32, 32, 3)
[INFO] label shape:  (3000,)
[INFO] labels[0]:  cats
[INFO] data shape:  (3000, 3072)
[INFO] features matrix: 9.0MB


In [89]:
# encode the labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# partition the data into training and testing splits using 
# 75% of the dtata for training and the remaining 25% for testing
(trainX, testX, trainY, testY)=train_test_split(data, labels, test_size=0.25, random_state=42)
print('[INFO] shape of trainX', trainX.shape)
print('[INFO] shape of trainY', trainY.shape)
print('[INFO] shape of testX', testX.shape)
print('[INFO] shape of testY', testY.shape)

[INFO] shape of trainX (2250, 3072)
[INFO] shape of trainY (2250,)
[INFO] shape of testX (750, 3072)
[INFO] shape of testY (750,)


In [90]:
# train and evaluate a k-NN classifier on the raw pixel intensities
print("[INFO] evaluating k-NN classifier ...")
model = KNeighborsClassifier(n_neighbors=args["neighbors"], n_jobs=args["jobs"])
model.fit(trainX, trainY)
print(classification_report(testY, model.predict(testX), target_names=le.classes_))

[INFO] evaluating k-NN classifier ...
              precision    recall  f1-score   support

        cats       0.38      0.53      0.44       262
        dogs       0.40      0.51      0.45       239
       panda       0.91      0.27      0.41       249

    accuracy                           0.44       750
   macro avg       0.56      0.44      0.44       750
weighted avg       0.56      0.44      0.44       750

